In [8]:
from bunkatopics import Bunka
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

#model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"
#model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_name = "OrdalieTech/Solon-embeddings-large-0.1"

embedding_model = HuggingFaceEmbeddings(model_name=model_name,# We recommend starting with a small model
                                        model_kwargs={"device": "cuda"}, # Or cuda if you have GPU
                                        encode_kwargs={"show_progress_bar": True}, # Show the progress of embeddings
                                        multi_process=False)  # set to True if you have mutliprocessing


In [1]:
import pandas as pd
#docs = pd.read_csv("https://raw.githubusercontent.com/regicid/gender_le_monde/main/test_bunka.csv")
df_docs = pd.read_csv("raw_data/lemonde_bunka.csv")

In [2]:
df_docs = df_docs.sample(1000, random_state=42)

In [3]:
df_docs.to_csv('raw_data/lemonde_bunka_test.csv')

In [4]:
print(len(df_docs))

sentences = list(df_docs['titles'])
print(len(sentences))

1000
1000


In [5]:
ids = list(df_docs['0'])

len(set(ids))

1000

In [6]:
# Just do the embedding

import string
import random
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.vectorstores.chroma import Chroma

characters = string.ascii_letters + string.digits
random_string = "".join(random.choice(characters) for _ in range(20))

df_loader = pd.DataFrame(sentences, columns=["text"])
df_loader["doc_id"] = ids

loader = DataFrameLoader(df_loader, page_content_column="text")
documents_langchain = loader.load()

# 3.48 minutes to load 3,000,000 millions documents in the DataFrameLoader of langchain

In [9]:
# Chroma = plus de 50 pour 3 millions d'embeddings

# the actual steps to embeddings
vectorstore = Chroma.from_documents(
    documents_langchain, embedding_model, collection_name=random_string
)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [11]:
bunka_ids = [item["doc_id"] for item in vectorstore.get()["metadatas"]]
bunka_docs = vectorstore.get()["documents"]


bunka_embeddings = vectorstore._collection.get(include=["embeddings"])[
    "embeddings"
]

In [18]:
import json

# Create a dictionary where IDs are keys and embeddings are values
embedding_dump = dict(zip(ids, bunka_embeddings))

with open('exports/embedding_lemonde.json', 'w') as json_file:
    json.dump(embedding_dump, json_file)

1000